In [ ]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [1]:
# cSpell:disable

import re

from devtools import debug
from langchain.output_parsers import PydanticOutputParser
from langchain_community.document_loaders.web_base import WebBaseLoader
from pydantic import BaseModel, Field

URL = "https://www.agroparistech.fr/formations-ingenieur/mention-biodiversite-ecologie-evolution-montpellier-parcours-m1-biodiversite-vegetale-gestion-ecosystemes-tropicaux-montpellier"

loader = WebBaseLoader(URL)
doc = loader.load()
page = doc[0].page_content.strip()
page_cleaned = re.sub(r"[\t ]+", " ", page)
page_cleaned = re.sub(r"\n{3,}", "\n\n", page_cleaned)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


class Parcours(BaseModel):
    metiers: list[str] = Field(
        description="liste de metiers auquels peut conduire la formation"
    )
    domaines: list[str] = Field(
        description="domaines auquels s'applique cette formation"
    )


system = """
    Extraire les metiers et les domaines d'une fiche de description de formation récupérée sur internet.  
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    --- 
    {fiche}
    --- 
    {format_instructions}"""

parser = PydanticOutputParser(pydantic_object=Parcours)

prompt = def_prompt(system=system, user=user).partial(
    format_instructions=parser.get_format_instructions()
)

llm = get_llm(temperature=0.0)
chain = prompt | llm | parser
summaries = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})

debug(summaries)

2024-07-17 17:04:52.861 | INFO     | python.ai_core.llm:get_llm:419 - get LLM:'gpt_35_edenai' -configurable: False - streaming: False


/tmp/ipykernel_643119/1225439580.py:36 <module>
    summaries: [
        Parcours(
            metiers=[
                'Chercheur en biodiversité',
                'Ecologue',
                'Spécialiste en gestion des écosystèmes tropicaux',
                'Consultant en environnement',
                'Responsable de projets de conservation',
                'Analyste en sciences du vivant',
                'Expert en biodiversité et écologie',
                'Analyste en gestion environnementale',
            ],
            domaines=[
                'Biodiversité végétale',
                'Ecosystèmes tropicaux',
                'Gestion environnementale',
                'Interdisciplinarité',
                'Conservation',
                'Plantes',
                'Sciences du vivant',
            ],
        ),
    ] (list) len=1


[Parcours(metiers=['Chercheur en biodiversité', 'Ecologue', 'Spécialiste en gestion des écosystèmes tropicaux', 'Consultant en environnement', 'Responsable de projets de conservation', 'Analyste en sciences du vivant', 'Expert en biodiversité et écologie', 'Analyste en gestion environnementale'], domaines=['Biodiversité végétale', 'Ecosystèmes tropicaux', 'Gestion environnementale', 'Interdisciplinarité', 'Conservation', 'Plantes', 'Sciences du vivant'])]

In [ ]:
print(summaries.)